In [ ]:
from cryptography.fernet import Fernet

# Генерация случайного ключа
key = Fernet.generate_key()
print("Key:", key)
cipher_suite = Fernet(key)

# Шифрование сообщения
message = """12345Текст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123
*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text
1ext 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст Text 123*-/екст 
Text 123*-/!678""".encode('utf-8')
encrypted_message = cipher_suite.encrypt(message)

# Расшифрование сообщения
decrypted_message = cipher_suite.decrypt(encrypted_message).decode('utf-8')

print("Original message:", message.decode('utf-8'))
print("Encrypted message:", encrypted_message)
print("Decrypted message:", decrypted_message)


In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

def split_message(message, chunk_size):
    """Разбивает сообщение на части фиксированного размера."""
    return [message[i:i + chunk_size] for i in range(0, len(message), chunk_size)]

def join_message(chunks):
    """Объединяет части сообщения в одно целое."""
    return b''.join(chunks)

def encrypt_chunks(chunks, public_key):
    """Шифрует каждую часть сообщения с использованием открытого ключа."""
    encrypted_chunks = []
    for chunk in chunks:
        encrypted_chunk = public_key.encrypt(
            chunk,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        encrypted_chunks.append(encrypted_chunk)
    return encrypted_chunks

def decrypt_chunks(encrypted_chunks, private_key):
    """Дешифрует каждую часть сообщения с использованием закрытого ключа."""
    decrypted_chunks = []
    for encrypted_chunk in encrypted_chunks:
        decrypted_chunk = private_key.decrypt(
            encrypted_chunk,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        decrypted_chunks.append(decrypted_chunk)
    return decrypted_chunks

# Генерация ключей
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# Получение открытого ключа
public_key = private_key.public_key()

# Исходное сообщение
message = b"Hello, World! This is a long message that will be split into chunks for encryption."

# Разбиваем сообщение на части
chunk_size = 64
message_chunks = split_message(message, chunk_size)

# Шифруем части сообщения
encrypted_chunks = encrypt_chunks(message_chunks, public_key)

# Дешифруем части сообщения
decrypted_chunks = decrypt_chunks(encrypted_chunks, private_key)

# Объединяем части в одно целое
decrypted_message = join_message(decrypted_chunks).decode('utf-8')

print("Original message:", message.decode('utf-8'))
print("Decrypted message:", decrypted_message)


In [ ]:
import vk_api
import requests

# ID сообщества, куда публиковать запись
group_id = '254488'

# Токен доступа VK API
access_token = 'vk1.a.poTJ---T18f-ShPn7dVsnUwRCmSlwO2t5ec8xXkcAngMRg'

vk_session = vk_api.VkApi(token=access_token)

vk = vk_session.get_api()

# Получение URL сервера для загрузки документа
upload_url = vk.docs.getUploadServer()['upload_url']

# Загрузка файла (документа) на сервер VK
file_path = 'image-1.png'
with open(file_path, 'rb') as file:
    files = {'file': (file_path, file)}
    response = requests.post(upload_url, files=files)
    upload_data = response.json()

document = vk.docs.save(file=upload_data['file'])

# Получение информации о загруженном документе
document_info = document['doc']

post_text = 'Текст записи'

# Публикация записи с прикрепленным документом
vk.wall.post(
    owner_id=f'-{group_id}',  # Указываем ID сообщества с минусом
    from_group=1,  # Публикация от имени группы
    message=post_text,
    attachments=f'doc{document_info["owner_id"]}_{document_info["id"]}'  # ID загруженного документа
)

print('Запись с прикрепленным документом опубликована успешно!')


In [ ]:
import vk_api

access_token = 'vk1.a.poTJ---T18f-ShPn7dVsnUwRCmSlwO2FTE8B1rt5ec8xXkcAngMRg'

vk_session = vk_api.VkApi(token=access_token)

# ID записи, из которой скачать файл
post_id = '-2224587828_9'  # Замените на нужный ID записи

# Получение информации о записи
response = vk_session.method('wall.getById', {'posts': post_id})

# Проверка, что запись с указанным ID существует
if response and 'items' in response and len(response['items']) > 0:
    post = response['items'][0]

    # Проверка, что в записи есть вложения
    if 'attachments' in post:
        attachments = post['attachments']
        
        # Поиск вложения типа "doc" (документ)
        for attachment in attachments:
            if attachment['type'] == 'doc':
                doc = attachment['doc']
                
                # Получение URL файла
                file_url = doc['url']
                
                # Скачивание файла
                response = vk_session.http.get(file_url)
                
                # Сохранение файла на диск
                with open('сохраненный_файл', 'wb') as file:
                    file.write(response.content)
                
                print('Файл успешно скачан и сохранен.')
                break
    else:
        print('Запись не содержит вложений.')
else:
    print('Запись с указанным ID не найдена.')
